In [17]:
import os
import pathlib

import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [2]:
min_nvisits = 3

In [3]:
allstar_file = pathlib.Path(
    '/mnt/home/apricewhelan/data/APOGEE_DR17/allStar-dr17-turbo20-beta.fits')
allvisit_file = pathlib.Path(
    '/mnt/home/apricewhelan/data/APOGEE_DR17/allVisit-dr17-turbo20-beta.fits')

calib_verr_file = pathlib.Path(
    '../cache/allVisit-dr17-turbo20-beta-calib-verr.fits')

In [4]:
allstar = at.Table.read(allstar_file)
allvisit = at.Table.read(allvisit_file)
verr = at.Table.read(calib_verr_file)

In [5]:
# Remove bad velocities / NaN / Inf values:
bad_visit_mask = (
    np.isfinite(allvisit['VHELIO']) &
    np.isfinite(allvisit['VRELERR']) &
    (allvisit['VRELERR'] < 100.) &
    (allvisit['VHELIO'] != -9999) &
    (np.abs(allvisit['VHELIO']) < 500.)
)
print(f"Filtered {len(bad_visit_mask) - bad_visit_mask.sum()} "
      "bad/NaN/-9999 visits")
allvisit = allvisit[bad_visit_mask]

Filtered 83857 bad/NaN/-9999 visits


In [7]:
starflag_bits = np.array([
    3,  # VERY_BRIGHT_NEIGHBOR
    16,  # SUSPECT_RV_COMBINATION
    18,  # BAD_RV_COMBINATION
    19,  # RV_REJECT
    22  # RV_FAIL
])
starflag_bitmask = np.sum(2**starflag_bits)

star_starflag_mask = (allstar['STARFLAG'] & starflag_bitmask) == 0
visit_starflag_mask = (allvisit['STARFLAG'] & starflag_bitmask) == 0

print(f"Using allstar STARFLAG bitmask {starflag_bitmask}), "
      f"filtered {len(allstar) - star_starflag_mask.sum()} sources")
print(f"Using allvisit STARFLAG bitmask {starflag_bitmask}), "
      f"filtered {len(allvisit) - visit_starflag_mask.sum()} visits")

Using allstar STARFLAG bitmask 5046280), filtered 11002 sources
Using allvisit STARFLAG bitmask 5046280), filtered 35159 visits


In [8]:
rvflag_bits = np.array([
    1,  # RV_BCFIT_FAIL
    3,  # RV_WINDOW_MASK
    4,  # RV_VALUE_ERROR
    5,  # RV_RUNTIME_ERROR
    6,  # RV_ERROR
    8,  # NO_GOOD_VISITS
    9,  # ALL_VISITS_REJECTED
    10  # RV_REJECT
])
rvflag_bitmask = np.sum(2**rvflag_bits)
rvflag_mask = (allvisit['RV_FLAG'] & rvflag_bitmask) == 0

print(f"Applying allvisit RVFLAG mask {rvflag_bitmask}, filtered "
      f"{len(allvisit) - rvflag_mask.sum()} visits")

# After quality and bitmask cut, figure out what APOGEE_IDs remain
allvisit = allvisit[visit_starflag_mask & rvflag_mask]
v_apogee_ids, counts = np.unique(allvisit['APOGEE_ID'],
                                 return_counts=True)
allstar_visit_mask = np.isin(allstar['APOGEE_ID'],
                             v_apogee_ids[counts >= min_nvisits])
print(f"Keeping only sources with > {min_nvisits} visits: filtered "
      f"{len(allstar_visit_mask) - allstar_visit_mask.sum()} sources")

Applying allvisit RVFLAG mask 1914, filtered 96 visits
Keeping only sources with > 3 visits: filtered 271610 sources


In [9]:
# TEFF_BAD, LOGG_BAD
aspcapflag_bits = [16, 17]

aspcapflag_val = np.sum(2 ** np.array(aspcapflag_bits))
aspcapflag_mask = (allstar['ASPCAPFLAG'] & aspcapflag_val) == 0
print(f"Using allstar ASPCAPFLAG bitmask {aspcapflag_val}, "
      f"filtered {len(allstar) - aspcapflag_mask.sum()}")

allstar = allstar[allstar_visit_mask &
                  star_starflag_mask &
                  aspcapflag_mask]

# Only load visits for stars that we're loading
allvisit = allvisit[np.isin(allvisit['APOGEE_ID'],
                            allstar['APOGEE_ID'])]
v_apogee_ids2 = np.unique(allvisit['APOGEE_ID'])
star_mask2 = np.isin(allstar['APOGEE_ID'], v_apogee_ids2)
allstar = allstar[star_mask2]

_, idx = np.unique(allstar['APOGEE_ID'], return_index=True)
allstar = allstar[idx]

allvisit = allvisit[np.isin(allvisit['APOGEE_ID'],
                            allstar['APOGEE_ID'])]

Using allstar ASPCAPFLAG bitmask 196608, filtered 19626


In [11]:
allvisit = at.join(allvisit, verr, keys='VISIT_ID')

Final check for min nvisits:

In [12]:
v_apogee_ids, counts = np.unique(allvisit['APOGEE_ID'],
                                 return_counts=True)
allstar_visit_mask = np.isin(allstar['APOGEE_ID'],
                             v_apogee_ids[counts >= min_nvisits])
allstar = allstar[allstar_visit_mask]
allvisit = allvisit[np.isin(allvisit['APOGEE_ID'],
                            allstar['APOGEE_ID'])]

In [14]:
print(f"{len(allstar)} unique stars left")
print(f"{len(allvisit)} unique visits left")

375824 unique stars left
2062369 unique visits left


In [15]:
assert np.all(np.unique(allvisit['APOGEE_ID'], return_counts=True)[1] >= 3)

In [21]:
colnames = [
    'APOGEE_ID',
    'TARGET_ID',
    'VISIT_ID',
    'FILE',
    'FIBERID',
    'CARTID',
    'PLATE',
    'MJD',
    'TELESCOPE',
    'SURVEY',
    'FIELD',
    'SNR',
    'STARFLAG',
    'STARFLAGS',
    'JD',
    'VREL',
    'VRELERR',
    'VHELIO',
    'BC',
    'N_COMPONENTS',
    'RV_COMPONENTS',
    'RV_FLAG',
    'CALIB_VERR'
]

In [ ]:
basename = os.path.splitext(allvisit_file.parts[-1])[0]
allvisit[colnames].write(
    f'../cache/{basename}-min{min_nvisits}-calibverr.fits',
    overwrite=True)